In [128]:
from abc import ABC, abstractmethod

class BaseValidator(ABC):
    required_class = None

    @abstractmethod
    def validate(self, value):
        pass

    def __set_name__(self, owner_class, prop_name):
        self.prop_name = prop_name

    def __set__(self, instance, value):
        if not isinstance(value, self.required_class):
            raise TypeError(f'{self.prop_name} must be of type {self.required_class.__name__}')
        self.validate(value)
        instance.__dict__[self.prop_name] = value

    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        else:
            return instance.__dict__.get(self.prop_name, None)
    

class IntegerField(BaseValidator):
    required_class = int

    def __init__(self, min_value=None, max_value=None):
        self.min_value = min_value
        self.max_value = max_value
        if (min_value is not None and max_value is not None) and min_value > max_value:
            raise ValueError('min_value cannot be more than max_value')

    def validate(self, value):
        if self.min_value is not None and value < self.min_value:
            raise ValueError(f'{self.prop_name} value must be more than {self.min_value}. Received {value}')
        if self.max_value is not None and value > self.max_value:
            raise ValueError(f'{self.prop_name} value must be less than {self.max_value}. Received {value}')

class CharField(BaseValidator):
    required_class = str

    def __init__(self, min_length=None, max_length=None):
        self.min_length = 0 if min_length is None else min_length
        self.max_length = 1000 if max_length is None else max_length
        if min_length is not None and min_length < 0:
            raise ValueError('min_length cannot be less than 0')
        if max_length is not None and max_length < 0:
            raise ValueError('max_length cannot be less than 0')
        if self.max_length < self.min_length:
            raise ValueError('max_length cannot be less than min_length')

    def validate(self, value):
        if len(value) < self.min_length or len(value) > self.max_length:
            raise ValueError(f'{self.prop_name} length must be between {self.min_length} and {self.max_length}. Received {value} with length: {len(value)}')


In [129]:
class Person:
    name = CharField()
    age = IntegerField(0, 100)

In [130]:
p = Person()

In [131]:
p.age = 100

In [132]:
p.age

100

In [133]:
p.name = 'LARARARARARARARARARARARARARARARARAR'

In [134]:
p.name

'LARARARARARARARARARARARARARARARARAR'

In [135]:
import unittest

def run_tests(test_class):
    suite = unittest.TestLoader().loadTestsFromTestCase(test_class)
    runner = unittest.TextTestRunner(verbosity=2)
    result = runner.run(suite)

In [143]:
class CharFieldTest(unittest.TestCase):
    
    def setUp(self):
        # Define Person class in setUp or create it fresh each time
        class TestPerson:
            name = CharField(0, 10)
        
        self.TestPerson = TestPerson
        self.p1 = TestPerson()
    
    def test_set_name(self):
        self.p1.name = 'Alex'

    def test_set_invalid_name(self):
        with self.assertRaises(ValueError):
            self.p1.name = 'Lalalalalalalaaaaaaaa'

    def test_invalid_descriptor(self):
        with self.assertRaises(ValueError):
            value = CharField(5, 2)

    def test_none_fields(self):
        value = CharField()
        self.assertEqual(value.min_length, 0)
        self.assertEqual(value.max_length, 1000)

class IntegerFieldTest(unittest.TestCase):
    def setUp(self):
        # Define Person class in setUp or create it fresh each time
        class TestPerson:
            age = IntegerField(0, 10)
        
        self.TestPerson = TestPerson
        self.p1 = TestPerson()
    
        def test_set_age(self):
            self.p1.age = 5

    def test_set_invalid_age(self):
        with self.assertRaises(ValueError):
            self.p1.age = 15

    def test_set_invalid_type(self):
        with self.assertRaises(TypeError):
            self.p1.age = 'String'

    def test_invalid_descriptor(self):
        with self.assertRaises(ValueError):
            value = IntegerField(5, 2)

    def test_none_fields(self):
        value = IntegerField()
        self.assertEqual(value.min_value, None)
        self.assertEqual(value.max_value, None)

In [144]:
run_tests(CharFieldTest)

test_invalid_descriptor (__main__.CharFieldTest.test_invalid_descriptor) ... ok
test_none_fields (__main__.CharFieldTest.test_none_fields) ... ok
test_set_invalid_name (__main__.CharFieldTest.test_set_invalid_name) ... ok
test_set_name (__main__.CharFieldTest.test_set_name) ... ok

----------------------------------------------------------------------
Ran 4 tests in 0.003s

OK


In [145]:
run_tests(IntegerFieldTest)

test_invalid_descriptor (__main__.IntegerFieldTest.test_invalid_descriptor) ... ok
test_none_fields (__main__.IntegerFieldTest.test_none_fields) ... ok
test_set_invalid_age (__main__.IntegerFieldTest.test_set_invalid_age) ... ok
test_set_invalid_type (__main__.IntegerFieldTest.test_set_invalid_type) ... ok

----------------------------------------------------------------------
Ran 4 tests in 0.004s

OK
